In [8]:
import pandas as pd
import numpy as np


In [3]:
import requests

#api_key='gaKMkjL1JzRhLGQ8pofFgbAN8wRMQNHcT0G96xr5'
from google.colab import userdata


search_query = 'cottage cheese 2%'
api_key = userdata.get('USDA_KEY')

# Step 1: Search for foods by keyword (e.g., "chicken")
search_url = "https://api.nal.usda.gov/fdc/v1/foods/search"
params = {
    'query': search_query,  # Search term
    'api_key': api_key
}

# Perform the search
response = requests.get(search_url, params=params)
data = response.json()



In [22]:
def USDA_APIRequest(api_key, search_query=None, fdc_id=None ):
  search_url = "https://api.nal.usda.gov/fdc/v1/foods/search"
  details_url = f"https://api.nal.usda.gov/fdc/v1/food/{fdc_id}"
  if api_key == None:
    print('no api key')
    data = {}
  else:
    if fdc_id == None:
      params = {
          'query': search_query,  # Search term
          'api_key': api_key
      }

      # Perform the search
      response = requests.get(search_url, params=params)
    else:
      response = requests.get(details_url, params={'api_key': api_key})

    data = response.json()

  return data

In [5]:
def QueryUSDA(query_block, api_key):
  query_list = query_block.split('\n')
  data_coll={}
  for query in query_list:
    print(query)
    search_url = "https://api.nal.usda.gov/fdc/v1/foods/search"
    params = {
        'query': query,  # Search term
        'api_key': api_key
    }

    # Perform the search
    response = requests.get(search_url, params=params)
    data = response.json()
    data_coll[query] = data
  return data_coll

In [23]:
USDA_data=USDA_APIRequest(search_query='cottage cheese 2% \n chicken', api_key=api_key)


In [27]:
for item in USDA_data:

  print(item, ":", USDA_data[item])

totalHits : 214649
currentPage : 1
totalPages : 4293
pageList : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
foodSearchCriteria : {'query': 'cottage cheese 2% \n chicken', 'generalSearchInput': 'cottage cheese 2% \n chicken', 'pageNumber': 1, 'numberOfResultsPerPage': 50, 'pageSize': 50, 'requireAllWords': False}
foods : [{'fdcId': 328841, 'description': 'Cheese, cottage, lowfat, 2% milkfat', 'commonNames': '', 'additionalDescriptions': '', 'dataType': 'Foundation', 'ndbNumber': 1015, 'publishedDate': '2019-04-01', 'foodCategory': 'Dairy and Egg Products', 'mostRecentAcquisitionDate': '2013-03-27', 'allHighlightFields': '', 'score': 776.1336, 'microbes': [], 'foodNutrients': [{'nutrientId': 1106, 'nutrientName': 'Vitamin A, RAE', 'nutrientNumber': '320', 'unitName': 'UG', 'derivationCode': 'NC', 'derivationDescription': 'Calculated', 'derivationId': 49, 'value': 69.0, 'foodNutrientSourceId': 2, 'foodNutrientSourceCode': '4', 'foodNutrientSourceDescription': 'Calculated or imputed', 'rank': 7420, 'i

In [9]:
# Extract the first food item
food_item = data['foods'][0]

# Extract the description
description = food_item['description']

# Extract the list of nutrients
food_nutrients = food_item['foodNutrients']
# Create a dictionary with nutrient names as keys and their values
nutrient_data = {nutrient['nutrientName']: nutrient['value'] for nutrient in food_nutrients}


df = pd.DataFrame(nutrient_data, index=[description])
df
# List of target nutrients
target_nutrients = ['Protein', 'Carbohydrate, by difference', 'Total lipid (fat)', 'Fiber, total dietary']
# Add missing nutrients with NaN values
for nutrient in target_nutrients:
    if nutrient not in df.columns:
        df[nutrient] = 0
columns = ['Ingredient']+['Calories']+target_nutrients
combined_df = pd.DataFrame(columns=columns)



In [10]:
# Define the target nutrients and columns for the final DataFrame
target_nutrients = ['Protein', 'Carbohydrate, by difference', 'Total lipid (fat)', 'Fiber, total dietary']
columns = ['Ingredient', 'Calories'] + target_nutrients

# Initialize empty DataFrame
combined_df = pd.DataFrame(columns=columns).set_index('Ingredient')

# Loop through USDA data
for item in USDA_data:
    food_item = USDA_data[item]['foods'][0]

    # Extract description and nutrients
    description = food_item['description']
    food_nutrients = food_item['foodNutrients']

    # Build nutrient dict
    nutrient_data = {n['nutrientName']: n['value'] for n in food_nutrients}

    # Ensure all target nutrients are present
    for nutrient in target_nutrients:
        if nutrient not in nutrient_data:
            nutrient_data[nutrient] = 0

    # Compute calories
    calories = (
        nutrient_data['Protein'] * 4 +
        nutrient_data['Carbohydrate, by difference'] * 4 +
        nutrient_data['Total lipid (fat)'] * 9 +
        nutrient_data['Fiber, total dietary'] * 2
    )

    # Construct row data
    row_data = {
        'Calories': calories,
        'Protein': nutrient_data['Protein'],
        'Carbohydrate, by difference': nutrient_data['Carbohydrate, by difference'],
        'Total lipid (fat)': nutrient_data['Total lipid (fat)'],
        'Fiber, total dietary': nutrient_data['Fiber, total dietary']
    }

    # Append row to DataFrame
    combined_df.loc[description] = row_data

# Show result
df = combined_df
df

,Calories,Protein,"Carbohydrate, by difference",Total lipid (fat),"Fiber, total dietary"
Ingredient,,,,,
"Cheese, cottage, lowfat, 2% milkfat",81.94,11.0,4.31,2.30,0.0
CHICKEN,101.71,21.4,0.00,1.79,0.0


In [11]:


# Rename columns
df.rename(columns={
    'Carbohydrate, by difference': 'Carbs',
    'Total lipid (fat)': 'Fat',
    'Fiber, total dietary': 'Fiber'
}, inplace=True)

# Calculate percent columns
df['% Protein'] = (df['Protein'] * 4) / df['Calories'] * 100
df['% Carbs']   = (df['Carbs']   * 4) / df['Calories'] * 100
df['% Fat']     = (df['Fat']     * 9) / df['Calories'] * 100
df['% Fiber']   = (df['Fiber']   * 2) / df['Calories'] * 100

# Round for readability (optional)
df[['% Protein', '% Carbs', '% Fat', '% Fiber']] = df[['% Protein', '% Carbs', '% Fat', '% Fiber']].round(1)

# Show final DataFrame
df


,Calories,Protein,Carbs,Fat,Fiber,% Protein,% Carbs,% Fat,% Fiber
Ingredient,,,,,,,,,
"Cheese, cottage, lowfat, 2% milkfat",81.94,11.0,4.31,2.30,0.0,53.7,21.0,25.3,0.0
CHICKEN,101.71,21.4,0.00,1.79,0.0,84.2,0.0,15.8,0.0


In [12]:
print(combined_df['Protein'].sum())

32.4


------------------------------------- individual query by fdc id ------------------------

In [13]:
# Step 2: Assume the user selects a row (e.g., row 1) from the list
# Select the first food item (or any other based on user input)
selected_food = data['foods'][0]  # In this case, selecting the first item
fdc_id = selected_food['fdcId']  # Get the FDC ID of the selected food

print(f"\nFetching nutrition details for: {selected_food['description']} (FDC ID: {fdc_id})")

# Step 3: Fetch detailed nutrition info using the FDC ID
details_url = f"https://api.nal.usda.gov/fdc/v1/food/{fdc_id}"
details_response = requests.get(details_url, params={'api_key': api_key})
food_details = details_response.json()

# Step 4: Display detailed nutrition info

print(f"Name: {food_details['description']}")




Fetching nutrition details for: Cheese, cottage, lowfat, 2% milkfat (FDC ID: 328841)
Name: Cheese, cottage, lowfat, 2% milkfat


In [14]:
# Step 2: Assume the user selects a row (e.g., row 1) from the list
# Select the first food item (or any other based on user input)
selected_food = data['foods'][0]  # In this case, selecting the first item
fdc_id = selected_food['fdcId']  # Get the FDC ID of the selected food

print(f"\nFetching nutrition details for: {selected_food['description']} (FDC ID: {fdc_id})")

# Step 3: Fetch detailed nutrition info using the FDC ID
details_url = f"https://api.nal.usda.gov/fdc/v1/food/{fdc_id}"
details_response = requests.get(details_url, params={'api_key': api_key})
food_details = details_response.json()

# Step 4: Display detailed nutrition info

print(f"Name: {food_details['description']}")




Fetching nutrition details for: Cheese, cottage, lowfat, 2% milkfat (FDC ID: 328841)
Name: Cheese, cottage, lowfat, 2% milkfat


In [15]:
food_nutrients

[{'nutrientId': 1003,
  'nutrientName': 'Protein',
  'nutrientNumber': '203',
  'unitName': 'G',
  'derivationCode': 'LCCS',
  'derivationDescription': 'Calculated from value per serving size measure',
  'derivationId': 70,
  'value': 21.4,
  'foodNutrientSourceId': 9,
  'foodNutrientSourceCode': '12',
  'foodNutrientSourceDescription': "Manufacturer's analytical; partial documentation",
  'rank': 600,
  'indentLevel': 1,
  'foodNutrientId': 25017824},
 {'nutrientId': 1004,
  'nutrientName': 'Total lipid (fat)',
  'nutrientNumber': '204',
  'unitName': 'G',
  'derivationCode': 'LCCS',
  'derivationDescription': 'Calculated from value per serving size measure',
  'derivationId': 70,
  'value': 1.79,
  'foodNutrientSourceId': 9,
  'foodNutrientSourceCode': '12',
  'foodNutrientSourceDescription': "Manufacturer's analytical; partial documentation",
  'rank': 800,
  'indentLevel': 1,
  'foodNutrientId': 25017825,
  'percentDailyValue': 2},
 {'nutrientId': 1005,
  'nutrientName': 'Carbohydr

In [16]:
food_details

{'fdcId': 328841,
 'description': 'Cheese, cottage, lowfat, 2% milkfat',
 'publicationDate': '4/1/2019',
 'foodNutrients': [{'nutrient': {'id': 2045,
    'number': '951',
    'name': 'Proximates',
    'rank': 50,
    'unitName': 'g'},
   'type': 'FoodNutrient'},
  {'type': 'FoodNutrient',
   'nutrient': {'id': 1051,
    'number': '255',
    'name': 'Water',
    'rank': 100,
    'unitName': 'g'},
   'foodNutrientDerivation': {'id': 1,
    'code': 'A',
    'description': 'Analytical',
    'foodNutrientSource': {'id': 1,
     'code': '1',
     'description': 'Analytical or derived from analytical'}},
   'id': 2248899,
   'amount': 81.1,
   'dataPoints': 16,
   'max': 81.9,
   'min': 80.2,
   'median': 81.2,
   'minYearAcquired': 2013,
   'nutrientAnalysisDetails': [{'subSampleId': 328645,
     'nutrientId': 1051,
     'nutrientAcquisitionDetails': [{'sampleUnitId': 328641,
       'purchaseDate': '3/22/2013',
       'storeCity': 'CARRBORO',
       'storeState': 'NC'},
      {'sampleUnitId'

In [17]:
unit_dict = {
    "g": 1,
    "mg": 0.001,
    "µg": 0.000001,
    "kcal": 0.00000001,
    "kJ": 0.00000001,
    "IU": 0.0000001
}

tot_amt=0
for nutrient in food_details['foodNutrients']:
    #print(nutrient)
    #print(nutrient['nutrient']['name'])

    if 'amount' in nutrient:
        nutrient_amt=nutrient['amount']
        nutrient_unit=nutrient['nutrient']['unitName']
        nutrient_name=nutrient['nutrient']['name']
        print(f"{nutrient_name}: {nutrient_amt} {nutrient_unit}")
        tot_amt += float(nutrient['amount']) * unit_dict[nutrient_unit]
        print(f"sub-total: {tot_amt}")

print(f"Total: {tot_amt}")


Water: 81.1 g
sub-total: 81.1
Energy: 84.0 kcal
sub-total: 81.10000083999999
Energy: 352.0 kJ
sub-total: 81.10000436
Nitrogen: 1.73 g
sub-total: 82.83000436
Protein: 11.0 g
sub-total: 93.83000436
Total lipid (fat): 2.3 g
sub-total: 96.13000436
Total fat (NLEA): 1.87 g
sub-total: 98.00000436
Ash: 1.25 g
sub-total: 99.25000436
Carbohydrate, by difference: 4.31 g
sub-total: 103.56000436000001
Sugars, Total: 4.1 g
sub-total: 107.66000436
Sucrose: 0.0 g
sub-total: 107.66000436
Glucose: 0.0 g
sub-total: 107.66000436
Fructose: 0.0 g
sub-total: 107.66000436
Lactose: 3.98 g
sub-total: 111.64000436
Maltose: 0.0 g
sub-total: 111.64000436
Galactose: 0.12 g
sub-total: 111.76000436000001
Calcium, Ca: 103.0 mg
sub-total: 111.86300436
Iron, Fe: 0.13 mg
sub-total: 111.86313436
Magnesium, Mg: 8.9 mg
sub-total: 111.87203436
Phosphorus, P: 148.0 mg
sub-total: 112.02003436
Potassium, K: 120.0 mg
sub-total: 112.14003436
Sodium, Na: 321.0 mg
sub-total: 112.46103436
Zinc, Zn: 0.61 mg
sub-total: 112.46164436
C